# Histórico do Schedule Management

Notebook a ponto de registrar o histórico de alterações do Schedule Management. Não é necessário executar este notebook.

## Get Instance

Como o Schedule Management será implementado como um Singleton, é necessário utilizar o método `get_instance` para obter a instância do objeto. Para isso, desenvolvemos dois testes, um para verificar a instância e outro para verificar se a instância é única:

In [ ]:
class TestScheduleManagement(unittest.TestCase):
    def setUp(self):
        # Reset the singleton instance before each test
        ScheduleManagement._instance = None
        self.db_module = Mock()
        
    def test_get_instance_creates_instance(self):
        instance = ScheduleManagement.get_instance(self.db_module)
        self.assertIsInstance(instance, ScheduleManagement)

    def test_get_instance_returns_same_instance(self):
        instance1 = ScheduleManagement.get_instance(self.db_module)
        instance2 = ScheduleManagement.get_instance(self.db_module)
        self.assertIs(instance1, instance2)

Para isso, o código abaixo foi desenvolvido:

In [ ]:
class ScheduleManagement:
    """
    ScheduleManagement class
    Responsible for managing the schedules in the database

    Attributes:
        db: Database module
        schedules: Dictionary of schedules, where the key is the schedule ID
            and the value is the schedule instance
    """

    _instance = None

    @classmethod
    def get_instance(cls, database_module: MongoModule, schedules: dict = None):
        if cls._instance is None:
            cls._instance = cls(database_module, schedules)
        return cls._instance

O código foi suficiente para verificar se a instância é única e se a instância é a mesma.

## Schedule Exists

Para verificar se um schedule existe, foi desenvolvido os seguintes testes:

In [ ]:
def test_schedule_exists(self):
    self.db_module.select_data = MagicMock(return_value=[{'_id': 'schedule1',\
                                                            'title': 'Schedule 1',\
                                                            'description': 'This is schedule 1',\
                                                            'permissions': {'user1': 'read', 'user2': 'write'},\
                                                            'elements': ['element1', 'element2']
                                                            }])
    result = self.schedule_management.schedule_exists('schedule1')
    self.assertTrue(result)
    self.db_module.select_data.assert_called_with('schedules', {'_id': 'schedule1'})

def test_schedule_does_not_exist(self):
    self.db_module.select_data = MagicMock(return_value=[])
    result = self.schedule_management.schedule_exists('schedule1')
    self.assertFalse(result)
    self.db_module.select_data.assert_called_with('schedules', {'_id': 'schedule1'})

O código não foi suficiente para verificar se o schedule existe, pois o método `schedule_exists` não foi implementado. O código abaixo foi desenvolvido para implementar o método:

In [ ]:
def schedule_exists(self, schedule_id: str) -> bool:
    """
    Check if a schedule exists

    Args:
        schedule_id: Schedule ID

    Returns:
        True if the schedule exists, False otherwise
    """

    # Get the schedules from the database that match the given ID
    schedule = self.db_module.select_data('schedules', {'_id': schedule_id})

    # If the list is not empty, the schedule exists
    return bool(schedule)

O código foi suficiente.